In [ ]:
import pandas as pd
import numpy as np
import torch


In [209]:
path = '../data/training/training_solutions_rev1.csv' #path è universale
dataframe = pd.read_csv(path, sep=',', index_col=False) 
df = dataframe.drop(columns='GalaxyID')
df

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.000000,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.000000,0.000000,0.0,0.325512
1,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.000000,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.000000,0.000000,0.0,0.000000
2,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,0.000000,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.000000,0.000000,0.0,0.000000
4,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61573,0.510379,0.489621,0.000000,0.059207,0.430414,0.000000,0.430414,0.226257,0.204157,0.043458,...,0.000000,0.226257,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.226257
61574,0.901216,0.098784,0.000000,0.000000,0.098784,0.000000,0.098784,0.000000,0.098784,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
61575,0.202841,0.777376,0.019783,0.116962,0.660414,0.067245,0.593168,0.140022,0.520391,0.000000,...,0.000000,0.000000,0.090673,0.049349,0.000000,0.067726,0.000000,0.000000,0.0,0.072296
61576,0.091000,0.909000,0.000000,0.045450,0.863550,0.022452,0.841098,0.795330,0.068220,0.000000,...,0.000000,0.068398,0.318132,0.408799,0.227464,0.408799,0.090668,0.023065,0.0,0.045334


In [210]:
def mapper(row,datafr=False):
    class1_3 = (row[:,0:3])
    class3_2 = (row[:,5:7])
    class4_2 = (row[:,7:9])
    class5_4 = (row[:,9:13])
    class7_3 = (row[:,15:18])
    class9_3 = (row[:,25:28])

    #normalizing class 4 and 5
    class4_2 = torch.where(class4_2.sum(dim=1, keepdim=True) != 0, 
                           class4_2 / class4_2.sum(dim=1, keepdim=True), 
                           class4_2)
    class5_4 = torch.where(class5_4.sum(dim=1, keepdim=True) != 0, 
                           class5_4 / class5_4.sum(dim=1, keepdim=True), 
                           class5_4)

    E0=class7_3[:,0]
    E3=class7_3[:,1]
    E6=class7_3[:,2]

    S0a_eon=class9_3[:,0]
    SB0a_eon=class9_3[:,1]
    Scd_eon=class9_3[:,2]

    SoB=class4_2[:,1]*class3_2[:,0]
    SoA=class4_2[:,1]*class3_2[:,1]

    SAa=class4_2[:,0]*class3_2[:,1]*class5_4[:,3]
    SAb=class4_2[:,0]*class3_2[:,1]*class5_4[:,2]
    SAc=class4_2[:,0]*class3_2[:,1]*class5_4[:,1]
    SAd=class4_2[:,0]*class3_2[:,1]*class5_4[:,0]

    SBa=class4_2[:,0]*class3_2[:,0]*class5_4[:,3]
    SBb=class4_2[:,0]*class3_2[:,0]*class5_4[:,2]
    SBc=class4_2[:,0]*class3_2[:,0]*class5_4[:,1]
    SBd=class4_2[:,0]*class3_2[:,0]*class5_4[:,0]

    A=class1_3[:,2]
    if datafr==True:x= pd.DataFrame({'E0':E0,
                     'E3': E3,
                     'E6': E6,
                     'S0a_eon': S0a_eon,
                     'SB0a_eon':SB0a_eon,
                     'Scd_eon': Scd_eon,
                     'SoB': SoB,
                     'SoA': SoA,
                     'SAa': SAa,
                     'SAb': SAb,
                     'SAc': SAc,
                     'SAd': SAd,
                     'SBa': SBa,
                     'SBb': SBb,
                     'SBc': SBc,
                     'SBd': SBd,
                     'A': A})
    else: x=torch.stack([E0,
                     E3,
                     E6,
                     S0a_eon,
                     SB0a_eon,
                     Scd_eon,
                     SoB,
                     SoA,
                     SAa,
                     SAb,
                     SAc,
                     SAd,
                     SBa,
                     SBb,
                     SBc,
                     SBd,
                     A],dim=1)
    return (x)
    


In [211]:
df_tensor=torch.Tensor(df.values)

In [212]:
x= mapper(df_tensor,datafr=True)
x

,E0,E3,E6,S0a_eon,SB0a_eon,Scd_eon,SoB,SoA,SAa,SAb,SAc,SAd,SBa,SBb,SBc,SBd,A
0,0.201463,0.181684,0.000000,0.000000,0.0,0.000000,0.012371,0.186084,0.000000,0.325695,0.066622,0.000000,0.000000,0.021652,0.004429,0.0,0.000000
1,0.000000,0.135082,0.191919,0.012414,0.0,0.018764,0.030491,0.010780,0.057345,0.039294,0.057810,0.000000,0.162208,0.111147,0.163523,0.0,0.009222
2,0.000000,0.741864,0.023853,0.000000,0.0,0.000000,0.000000,0.177352,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.056931
3,0.408599,0.284778,0.000000,0.000000,0.0,0.000000,0.022703,0.026763,0.053726,0.048582,0.000000,0.000000,0.045577,0.041213,0.000000,0.0,0.068059
4,0.494587,0.439252,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.066161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61573,0.000000,0.410816,0.099563,0.059207,0.0,0.000000,0.000000,0.204157,0.066314,0.000000,0.137097,0.022845,0.000000,0.000000,0.000000,0.0,0.000000
61574,0.404841,0.496375,0.000000,0.000000,0.0,0.000000,0.000000,0.098784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
61575,0.030477,0.172364,0.000000,0.116962,0.0,0.000000,0.052988,0.467404,0.010656,0.066143,0.048966,0.000000,0.001208,0.007498,0.005551,0.0,0.019783
61576,0.045500,0.045500,0.000000,0.045450,0.0,0.000000,0.001774,0.066447,0.081338,0.489579,0.203733,0.000000,0.002171,0.013069,0.005438,0.0,0.000000


In [213]:
x['sum']=x.sum(axis=1)
x['sum']

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
61573    1.0
61574    1.0
61575    1.0
61576    1.0
61577    1.0
Name: sum, Length: 61578, dtype: float32